# Open-Source Formal Verification for Chisel

In this notebook, we illustrate some of the new formal verification features that are the topic of our [WOSET 2021 paper](https://woset-workshop.github.io/WOSET2021.html#article-3).

First we download the latest [Chisel](https://github.com/chipsalliance/chisel3) and [chiseltest](https://github.com/ucb-bar/chisel-testers2) snapshot release from the Sonatypes package repository:

In [ ]:
interp.repositories() ++= Seq(
  coursierapi.MavenRepository.of("https://oss.sonatype.org/content/repositories/snapshots")
)

In [ ]:
import $ivy.`edu.berkeley.cs::chisel3:3.5-SNAPSHOT`
import $ivy.`edu.berkeley.cs::chiseltest:0.5-SNAPSHOT`

In [ ]:
import chisel3._
import chisel3.stage._
import firrtl.options.Dependency
import firrtl.stage.RunFirrtlTransformAnnotation

Now we can declare a simple Chisel module:

In [ ]:
class KeepMax(width: Int) extends Module {
  val in = IO(Input(UInt(width.W)))
  val out = IO(Output(UInt(width.W)))

  val max = RegInit(0.U(width.W))
  when (in > max) {
    max := in
  }
  out := max
    
  // get the value of io.out from 1 cycle in the past
  val firstCycle = RegInit(true.B)
  firstCycle := false.B
  when(!firstCycle) {
    assert(out >= RegNext(out))
  }
}

## Compilation Flow
Chisel is a Scala library for RTL circuits. These circuits can be exported into HiFirrtl which represents a static RTL circuit with high level Chisel features like memories, bundles and vectors. The firrtl compiler lowers these features to a more simple LoFirrtl representation which can then be exported to Verilog.

In [ ]:
val compiler = new ChiselStage
val genKeepMax = ChiselGeneratorAnnotation(() => new KeepMax(8))

In [ ]:
// compile to optimized low firrtl
compiler.execute(Array("-E", "low-opt"), Seq(genKeepMax))

Go ahead and inspect the new `KeepMax.opt.lo.fir` file that was generated in the root directory. We can see how the `assert` is represented as a native firrtl statement with two input: enable and predicate.

```
assert(clock, geq(out, REG), and(not(firstCycle), not(reset)), "") : assert
```

The `assert` statement is triggered on every rising edge of the `clock` signal. It is only enabled when it is not the `firstCycle` and when `reset` is not active. If that is the case, then it will check that `out` is greater or equal to the value of the delay register (`REG`) that was inserted by `RegNext(out)`.

There is also a `printf` statement which gets added by the `chisel3.assert` function. This statement will be ignored by our formal tools. It is useful in simulation and since we are going to use a simulator to replay the counter example that a formal tool might provide to us, it will also be useful to debug the failure of a formal check.

## Compilation to Verilog
LoFirrtl is normally compiled down to a simple subset of the (System)Verilog standard which works with a wide range of synthesis and verification tools. Thus we could compile our fomral property down to a concurrent assertion which works with both [Verilator](https://github.com/verilator/verilator/) as well as [SymbiYosys](https://github.com/YosysHQ/SymbiYosys).

In [ ]:
// compile to system verilog
compiler.execute(Array("-E", "sverilog"), Seq(genKeepMax))

In [ ]:
// generate SMTLib using yosys from SystemVerilog
os.proc("yosys", "-p", "read_verilog -sv KeepMax.sv ; proc ; write_smt2 KeepMax.smt2").call(stdout = os.Inherit, stderr = os.Inherit)

Have a look at the new `KeepMax.sv` and `KeepMax.smt2` files.

## Modelling Undefined Values and Compiling Directly to SMTLib2 and Btor2

There are some situations where the result of a Chisel circuit is undefined. The most straight forward way to create an arbitrary value is to use the `DontCare` construct.
By default the compiler will often replace abirary values with concrete values that will help it optimize the circuit. For formal verification however, we want these
arbitrary values to be treated in an "adverserial" manner, meaning the solver is free to choose a value that will make a property fail.
For this purpose we added a new `DefRandom` statement and passes to model arbitrary values to the firrtl compiler.

In [ ]:
// since `a` is assigned an arbitrary value, the assertion will fail, no matter what value we compare against
class InvalidSignalIs(value: Int) extends Module {
  val a = Wire(UInt(2.W))
  a := DontCare
  assert(a === value.U)
}

In [ ]:
// We expect the assertion to fail, however, when we compile this example to optimized firrtl, we end up with an `asert(true)`.
// The reason why this happens is that the compiler will opportunistically assign `a` to zero and thus the assertion is always true.
compiler.emitFirrtl(new InvalidSignalIs(0), Array("-E", "low-opt"))

In [ ]:
// When we compare to a different value, we get the expected `assert(false)`:
compiler.emitFirrtl(new InvalidSignalIs(1), Array("-E", "low-opt"))

In [ ]:
// We can use our new `InvalidToRandom` pass to replace all `DontCare` expressions with "random" inputs before the compiler starts optimizing:
import firrtl.backends.experimental.smt.random._
import firrtl.backends.experimental.smt._

compiler.emitFirrtl(new InvalidSignalIs(1), Array("-E", "low-opt"), Seq(RunFirrtlTransformAnnotation(Dependency(InvalidToRandomPass))))

In [ ]:
// the firrtl SMT backend will model the random signal as an inpupt
val r = compiler.execute(Array("-E", "experimental-smt2"), Seq(ChiselGeneratorAnnotation(() => new InvalidSignalIs(1)), RunFirrtlTransformAnnotation(Dependency(InvalidToRandomPass))))
r.collectFirst { case TransitionSystemAnnotation(s) => s }.get.serialize

In [ ]:
// the common transition system representation can be serialized as SMTLib2 (in a yosys derived encoding) or btor2 file by selecting the desired emitter (`-E`)
compiler.execute(Array("-E", "experimental-smt2", "-E", "experimental-btor2"), Seq(ChiselGeneratorAnnotation(() => new InvalidSignalIs(1)), RunFirrtlTransformAnnotation(Dependency(InvalidToRandomPass))))

Have a look at the `InvalidSignal.btor2` and `InvalidSignal.smt2` files the we generated and compare them to the internal Transition System representation that we printed out above.
We tried our best to keep signal names and annotations alive throughout the translation process.

Transition System:
```
InvalidSignalIs
input reset : bv<1>
input a_invalid : bv<2>
node a_invalid_cond : bv<1> = 1'b1
output a_invalid.en : bv<1> = a_invalid_cond
node a : bv<2> = ite(a_invalid_cond, a_invalid, 2'b0)
node _T : bv<1> = eq(a, zext(1'b1, 1))
node _T_1 : bv<1> = reset[0]
node _T_2 : bv<1> = eq(_T_1, 1'b0)
node _T_3 : bv<1> = eq(_T, 1'b0)
bad assert : bv<1> = not(implies(_T_2, _T))
```

[botr2](https://link.springer.com/content/pdf/10.1007%2F978-3-319-96145-3_32.pdf) encoding:
```
; BTOR description generated by firrtl 1.5-SNAPSHOT for module InvalidSignalIs.
1 sort bitvec 1
2 input 1 reset
3 sort bitvec 2
4 input 3 a_invalid
5 one 1
6 output 5 ; a_invalid.en
7 zero 3
8 ite 3 5 4 7
9 one 1
10 uext 3 9 1
11 eq 1 8 10 ; @[cmd18.sc 4:12] @[cmd18.sc 4:9]
12 zero 1
13 eq 1 2 12 ; @[cmd18.sc 4:9]
14 zero 1
15 eq 1 11 14 ; @[cmd18.sc 4:9]
16 implies 1 13 11
17 not 1 16
18 bad 17 ; assert @[cmd18.sc 4:9]
```

In [ ]:
// Now that we have seen a very simple example translated, let us generate the btor2 description of our KeepMax circuit once using yosys and once directly from the firrtl compiler:
compiler.execute(Array("-E", "sverilog"), Seq(genKeepMax))
os.proc("yosys", "-p", "read_verilog -sv KeepMax.sv ; proc ; write_btor KeepMax.yo.btor2").call(stdout = os.Inherit, stderr = os.Inherit)
compiler.execute(Array("-E", "experimental-btor2"), Seq(genKeepMax))

Have a look at the newly created `KeepMax.yo.btor2` (Chisel -> firrtl -> Verilog -> yosys -> btor2) and `KeepMax.btor2` (Chisel -> firrtl -> btor2).
They should be fairly similar since the `KeepMax` circuit is not relying on any undefined/arbitrary values in Chisel.